The MNIST digits data set has been studied for a long time, so researcher have created more challenging alternatives to benchmark modern machine learning algorithms.

We're going to create an image recognizer for the [Fashion MNIST data set](https://github.com/zalandoresearch/fashion-mnist)

The data CSVs can be downloaded [here](https://www.kaggle.com/zalando-research/fashionmnist/downloads/fashionmnist.zip/4)

Import the needed libraries

In [ ]:
import pandas as pd    # data formatting
import numpy as np     # numeric library
from sklearn.metrics import confusion_matrix
import random
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
%matplotlib inline
from matplotlib import pylab, pyplot  # plotting

Import the data from csv files into Pandas DataFrames. A DataFrame is similar to an Excel spreadsheet; it is made up of columns and rows.

In [ ]:
train = pd.read_csv('fashion-mnist_train.csv', header=0)
test = pd.read_csv('fashion-mnist_test.csv', header=0)

Let's see what the data looks like

In [ ]:
print(train.shape)
print(test.shape)

The dataframes have 785 columns. Each row corresponds to 1 image. In the 0th column is a label (0-9) saying what image it is. The other 784 columns each represent a single pixel.

The 28x28 square of the image has been unrolled (or reshaped) into a single long row 1x784.

In [ ]:
train.head()

In [ ]:
test.describe()

In [ ]:
label_name = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

Let's visualize a single row:

In [ ]:
row = 9
print("this image is", label_name[train.iloc[row,0]])
image = train.iloc[row, 1:].values.reshape(28,28)
pylab.imshow(image, cmap='gray')

Normalize pixel values to be between 0 and 1

In [ ]:
X_train = train.iloc[:,1:]/255
X_test = test.iloc[:,1:]/255

Convert labels to categories

In [ ]:
y_train = keras.utils.to_categorical(train.label, num_classes=10)
y_test = keras.utils.to_categorical(test.label, num_classes=10)

Compare labels and categories

In [ ]:
print('labels:')
print(train.label[:5].values)
print('corresponding categories:')
print(y_train[:5])

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Choose Sequential model - simple stack of layers

In [ ]:
model = Sequential()

Add hidden layer.

"Dense" means every node in one layer is connected to every node in other layer.

"Activation" is how node value is calculated from previous layer nodes and connection weights.

In [ ]:
layer_input = Dense(units=100,
                    activation='relu', 
                    input_shape=(784,)) 
model.add(layer_input)

Additional hidden layer

In [ ]:
model.add(Dense(units=50, activation='relu'))

Add output layer.

"Softmax" activation produces probabilities.

In [ ]:
layer_output = Dense(units=10,
                     activation='softmax')

model.add(layer_output)

Look at model architecture

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
%%time
training = model.fit(X_train, 
                     y_train,
                     epochs=10, # Number of passes over complete dataset
                     verbose=True, 
                     validation_split=0.1)

In [ ]:
predictions = model.predict_classes(X_test)

In [ ]:
labels = test.label.values

In [ ]:
predictions[:10]

In [ ]:
labels[:10]

In [ ]:
accuracy = sum(predictions == labels)/len(predictions)
print(accuracy)

Which ones are we getting wrong?

In [ ]:
wrong = np.where(predictions != labels)

In [ ]:
wrong

Randomly select the index of one of the numbers we got wrong

In [ ]:
wrong_index = random.choice(wrong[0])

print(
    "predicted: {}, label: {}, index: {}".format(
        label_name[predictions[wrong_index]], 
        label_name[labels[wrong_index]], 
        wrong_index
    )
)

image = test.iloc[wrong_index,1:].values.reshape(28,28)
pylab.imshow(image, cmap='gray')

Confusion Matrix: See how data is labeled and mislabeled, by category.

In [ ]:
cm = confusion_matrix(labels, predictions)
pyplot.matshow(cm)
pyplot.title('Confusion matrix')
pyplot.colorbar()
pyplot.ylabel('True label')
pyplot.xlabel('Predicted label')
pyplot.show()

Since we're usually right, it's hard to see what we're getting wrong (shades of blue). Let's ignore the cases where we are right:

In [ ]:
np.fill_diagonal(cm, 0)
pyplot.matshow(cm)
pyplot.title('Confusion matrix')
pyplot.colorbar()
pyplot.ylabel('True label')
pyplot.xlabel('Predicted label')
pyplot.show()